In [31]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)



def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f

def divisor_set_general(q,p):
    divisor = set()
    for d in range(2, q):
        if q%d ==1:
            divisor.add(d)
    for d in range(2,p):
        if p%d==1 and d !=q:
            divisor.add(d)
    for d in range(2,p+q+1):
        if (q*p+1)%d==0 and (p+q)%d==0:
            divisor.add(d)
    return divisor 
   
    
def cyclotomic_factor(q,p):
    """
    return the cyclotmic factor of F_{qp}
    """
    cyc_factor =x
    divisor_set = divisor_set_general(q,p)
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor


def fekete(q,p):
    n=p*q
    cyc_factor = cyclotomic_factor(q,p)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 


def reduced_fekete(f):
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        


def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 

def search_two_four_cycle_2(f,m,n):
    for q in range(m,n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 



def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result

def is_discriminant_a_square(q,p):
    if p % q ==1:
        return False
    else:
        if p % 4 == 1 and q % 4 ==1:
            return False
    return True    

# Galois group for $g_{n}$ with $n = pq$ and $n<1000$. 

We remark that since we already dealt with the case $q \in \{3, 5, 7, 11 \}$ separately, we will only consider $q \geq 13$. 

In [18]:
q_list = [q for q in srange(2, 100) if is_prime(q) and q>11]
print(q_list)

[13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


The following lines of code find the triple $(q_1, q_2, q_3)$ that guarantees that the Galois group of $g_n$ is $S_m$ where $m = \deg(g_n)$.

In [23]:
%%time
P=Primes()
N=10**5
q=13
p=q
for q in q_list: 
    p=q 
    while p*q < 1000: 
        p=P.next(p)
        f=fekete(q,p)
        g = reduced_fekete(f)
        print([p*q, q, p], triple(g,N))

[221, 13, 17] [449, 719, 193]


[247, 13, 19] [761, 1993, 191]


[299, 13, 23] [19, 503, 157]


[377, 13, 29] [313, 223, 241]


[403, 13, 31] [2969, 1021, 167]


...

[989, 23, 43] [107, 3931, 83]


[1081, 23, 47] [563, 2609, 173]


[899, 29, 31] [1213, 8273, 269]


[1073, 29, 37] [1873, 1399, 59]


[1147, 31, 37] [19, 15173, 103]
CPU times: user 16min 11s, sys: 4.75 s, total: 16min 16s
Wall time: 17min 16s


The following lines of code find the prime $q$ required to compute the Galois group of $f_n$. We remark that there are two cases to consider depending on whether the discriminant of $f_n$ is a square or not. 

In [41]:
P=Primes()
N=10**5
for q in q_list: 
    p=q 
    while p*q < 1000:
        p = P.next(p)
        f = fekete(q,p)
        if is_discriminant_a_square(q,p):
            print([p*q, q,p], search_two_four_cycle(f,N))
        else:
            print([p*q, q, p],two_cycle(f,N) )
            


[221, 13, 17] 1439


[247, 13, 19] 857


[299, 13, 23] 401


[377, 13, 29] 1427


[403, 13, 31] 4523


...

[989, 23, 43] 101


[1081, 23, 47] 2609


[899, 29, 31] 11113


[1073, 29, 37] 1399


[1147, 31, 37] 15131


# Galois group for $g_{n}$ with $n = pq$ and $1000<n<1500$. 


In [43]:
%%time
P=Primes()
N=10**5
q=13
p=q
for q in q_list: 
    p= 1000//q
    while p*q < 1100 and q<p: 
        p= P.next(p)
        f=fekete(q,p)
        g = reduced_fekete(f)
        print([p*q, q, p], triple(g,N))

[1027, 13, 79] [6703, 8999, 281]


[1079, 13, 83] [359, 2521, 61]


[1157, 13, 89] [4861, 6197, 31]


[1003, 17, 59] [2699, 1607, 359]


[1037, 17, 61] [317, 5227, 353]


...

[1081, 23, 47] [563, 2609, 173]


[1219, 23, 53] [1429, 1993, 5]


[1073, 29, 37] [1873, 1399, 59]


[1189, 29, 41] [7621, 7109, 83]


[1147, 31, 37] [19, 15173, 103]
CPU times: user 11min 46s, sys: 3.4 s, total: 11min 49s
Wall time: 12min 10s


In [44]:
%%time
P=Primes()
N=10**5
for q in q_list: 
    p= 1100//q
    while p*q < 1500 and q<p: 
        p= P.next(p)
        f=fekete(q,p)
        g = reduced_fekete(f)
        print([p*q, q, p], triple(g,N))

[1157, 13, 89] [4861, 6197, 31]


[1261, 13, 97] [5839, 1301, 211]


[1313, 13, 101] [4987, 601, 71]


[1339, 13, 103] [12347, 71, 887]


[1391, 13, 107] [4271, 6007, 881]


...

[1147, 31, 37] [19, 15173, 103]


[1271, 31, 41] [11, 5563, 239]


[1333, 31, 43] [10729, 1439, 359]


[1457, 31, 47] [3323, 929, 463]


[1643, 31, 53] [6481, 167, 241]
CPU times: user 52min 30s, sys: 13.3 s, total: 52min 43s
Wall time: 55min 11s


# Galois group for $f_{n}$ with $n = pq$ and $1000<n<1500$. 


In [45]:
P=Primes()
N=10**5
for q in q_list: 
    p= 1000//q 
    while p*q < 1500 and q<p:
        p = P.next(p)
        f = fekete(q,p)
        if is_discriminant_a_square(q,p):
            print([p*q, q,p], search_two_four_cycle(f,N))
        else:
            print([p*q, q, p],two_cycle(f,N) )
            


[1027, 13, 79] 109


[1079, 13, 83] 21157


[1157, 13, 89] 23623


[1261, 13, 97] 15427


[1313, 13, 101] 13679


...

[1147, 31, 37] 15131


[1271, 31, 41] 2711


[1333, 31, 43] 18637


[1457, 31, 47] 21773


[1643, 31, 53] 2477
